# MY TOP 3

* * *

**Difficulty:** Medium

**Proposition:** Retrieve customer details with more than 5 orders.

**Table:** Sales.Customer, Sales.Order

**Columns:** CustomerID, CustomerCompanyName, and NumberOfOrders from a derived table CustomerOrders that joins Sales.Customer and Sales.Order.

**Predicate:**

```
WHERE NumberOfOrders > 5

```

This WHERE clause filters the derived table CustomerOrders to only return rows where the NumberOfOrders is greater than 5.

**Why did I choose it?**

- It was a database I was most familiar with, since I've used it most of the semester so far
    
- It mimics how a ecommerce queries would look
    
- The CTE was useful for breaking down the query into simpler, more readable part by performing the join and aggregation to count orders per customer upfront.

In [ ]:
USE Northwinds2022TSQLV7;
WITH
  CustomerOrders
  AS
  (
    SELECT
      c.CustomerID,
      c.CustomerCompanyName,
      COUNT(o.OrderID) AS NumberOfOrders
    FROM
      Sales.Customer c
      JOIN Sales.[Order] o ON c.CustomerID = o.CustomerID
    GROUP BY
    c.CustomerID, c.CustomerCompanyName
  )
SELECT
  CustomerID,
  CustomerCompanyName,
  NumberOfOrders
FROM
  CustomerOrders
WHERE
  CustomerID IN (
    SELECT
  CustomerID
FROM
  CustomerOrders
WHERE
      NumberOfOrders > 5
  );
  

--JSON
USE Northwinds2022TSQLV7;
WITH
  CustomerOrders
  AS
  (
    SELECT
      c.CustomerID,
      c.CustomerCompanyName,
      COUNT(o.OrderID) AS NumberOfOrders
    FROM
      Sales.Customer c
      JOIN Sales.[Order] o ON c.CustomerID = o.CustomerID
    GROUP BY
    c.CustomerID, c.CustomerCompanyName
  )
SELECT
  CustomerID,
  CustomerCompanyName,
  NumberOfOrders
FROM
  CustomerOrders
WHERE
  CustomerID IN (
    SELECT
  CustomerID
FROM
  CustomerOrders
WHERE
      NumberOfOrders > 5
  )
FOR JSON PATH, ROOT('CustomerOrdersDetails');



***
**Difficulty:** Medium

**Proposition:** Aggregate average costs and repairs costs per car model, including the total stock count, ordered by stock count in descending order.

**Table:** Data.Model, Data.Stock

**Columns:**

Data.Model: ModelID, ModelName
Data.Stock: ModelID, Cost, RepairsCost

**Predicate:**

```
(SELECT COUNT(*) FROM Data.Stock WHERE ModelID = m.ModelID) AS TotalStockCount
```

This subquery calculates the total stock count for each model by filtering the Data.Stock table based on the ModelID.


**Why did I choose it?**

- This query memics a car dealership and provides information about the inventory of car models, focusing on the average costs and repairs costs, along with the total stock count

- This query also provides real life example of how to provide the financial health and inventory management of a car dealership

In [ ]:
USE PrestigeCars;
GO

WITH
    ModelCosts
    AS
    (
        SELECT
            ModelID,
            AVG(Cost) AS AvgCost,
            AVG(RepairsCost) AS AvgRepairsCost
        FROM Data.Stock
        GROUP BY ModelID
    )

SELECT
    m.ModelName,
    mc.AvgCost,
    mc.AvgRepairsCost,
    (SELECT COUNT(*)
    FROM Data.Stock
    WHERE ModelID = m.ModelID) AS TotalStockCount
FROM Data.Model m
    INNER JOIN ModelCosts mc ON m.ModelID = mc.ModelID
ORDER BY TotalStockCount DESC;

--JSON
USE PrestigeCars;
GO

WITH
    ModelCosts
    AS
    (
        SELECT
            ModelID,
            AVG(Cost) AS AvgCost,
            AVG(RepairsCost) AS AvgRepairsCost
        FROM Data.Stock
        GROUP BY ModelID
    )

SELECT
    m.ModelName,
    mc.AvgCost,
    mc.AvgRepairsCost,
    (SELECT COUNT(*)
    FROM Data.Stock
    WHERE ModelID = m.ModelID) AS TotalStockCount
FROM Data.Model m
    INNER JOIN ModelCosts mc ON m.ModelID = mc.ModelID
ORDER BY TotalStockCount DESC
FOR JSON PATH, ROOT('SalesCustomerCategory');


* * *

**Difficulty:** Complex

**Proposition:** Calculate the average sales value per employee-customer pair within a specified timeframe using a user-defined function for average calculation.

**Function:** dbo.CalculateAverageSale

**Parameters:**

@TotalSalesAmount as MONEY (total sales amount for the pair), @NumberOfSales as INT (number of sales transactions).

**Returns:** The average sales value as MONEY. If there are sales, it calculates the average; otherwise, returns 0.

**Tables and Columns:**

Sales.\[Order\]: EmployeeId, CustomerId, Freight (as sales value), OrderId, OrderDate HumanResources.Employee: EmployeeId, EmployeeFirstName, EmployeeLastName Sales.Customer: CustomerId, CustomerCompanyName

**Predicate:**

```
WHERE o.OrderDate BETWEEN '2014-01-01' AND '2014-12-31

```

This clause filters orders to the year 2014, targeting sales analysis within this period.

**Why did I choose it?**
- The query shows a employee-customer sale relationship

- The scalar function enhances the readability and maintainability of the code, allowing for the average sale calculation to be easily used in other queries or reports without duplicating code

- The query provides business insights



In [ ]:

USE Northwinds2022TSQLV7;
GO
CREATE OR ALTER FUNCTION dbo.CalculateAverageSale
(
    @TotalSalesAmount MONEY,
    @NumberOfSales INT
)
RETURNS MONEY
AS
BEGIN
    RETURN CASE WHEN @NumberOfSales > 0 THEN @TotalSalesAmount / @NumberOfSales ELSE 0 END;
END;
GO
;WITH
    EmployeeCustomerSales
    AS
    (
        SELECT
            o.EmployeeId,
            o.CustomerId,
            SUM(o.Freight) AS TotalSalesValue,
            COUNT(o.OrderId) AS NumberOfSales
        FROM
            Sales.[Order] o
        WHERE
        o.OrderDate BETWEEN '2014-01-01' AND '2014-12-31'
        GROUP BY
        o.EmployeeId,
        o.CustomerId
    )

SELECT
    e.EmployeeFirstName,
    e.EmployeeLastName,
    c.CustomerCompanyName,
    ECS.TotalSalesValue,
    dbo.CalculateAverageSale(ECS.TotalSalesValue, ECS.NumberOfSales) AS AverageSaleValue
FROM
    EmployeeCustomerSales ECS
    INNER JOIN
    HumanResources.Employee e ON ECS.EmployeeId = e.EmployeeId
    INNER JOIN
    Sales.Customer c ON ECS.CustomerId = c.CustomerId
GROUP BY
    e.EmployeeFirstName,
    e.EmployeeLastName,
    c.CustomerCompanyName,
    ECS.TotalSalesValue,
    ECS.NumberOfSales
ORDER BY
    e.EmployeeLastName,
    c.CustomerCompanyName;
    
--JSON
USE Northwinds2022TSQLV7;
GO
CREATE OR ALTER FUNCTION dbo.CalculateAverageSale
(
    @TotalSalesAmount MONEY,
    @NumberOfSales INT
)
RETURNS MONEY
AS
BEGIN
    RETURN CASE WHEN @NumberOfSales > 0 THEN @TotalSalesAmount / @NumberOfSales ELSE 0 END;
END;
GO
;WITH
    EmployeeCustomerSales
    AS
    (
        SELECT
            o.EmployeeId,
            o.CustomerId,
            SUM(o.Freight) AS TotalSalesValue,
            COUNT(o.OrderId) AS NumberOfSales
        FROM
            Sales.[Order] o
        WHERE
        o.OrderDate BETWEEN '2014-01-01' AND '2014-12-31'
        GROUP BY
        o.EmployeeId,
        o.CustomerId
    )

SELECT
    e.EmployeeFirstName,
    e.EmployeeLastName,
    c.CustomerCompanyName,
    ECS.TotalSalesValue,
    dbo.CalculateAverageSale(ECS.TotalSalesValue, ECS.NumberOfSales) AS AverageSaleValue
FROM
    EmployeeCustomerSales ECS
    INNER JOIN
    HumanResources.Employee e ON ECS.EmployeeId = e.EmployeeId
    INNER JOIN
    Sales.Customer c ON ECS.CustomerId = c.CustomerId
GROUP BY
    e.EmployeeFirstName,
    e.EmployeeLastName,
    c.CustomerCompanyName,
    ECS.TotalSalesValue,
    ECS.NumberOfSales
ORDER BY
    e.EmployeeLastName,
    c.CustomerCompanyName
FOR JSON PATH, ROOT('AverageSales');

# MY WORST 3

* * *

**Difficulty:** Medium

**Proposition:** Write a query that retrieves each business entity's total email addresses and phone numbers.

**Table:** Person.EmailAddress, Person.PersonPhone, Person.BusinessEntityContact

**Columns:** BusinessEntityID, EmailTotal (total email addresses), and PhoneTotal (total phone numbers) for each business entity.

**Predicate:**

```
-- For EmailTotal
GROUP BY BusinessEntityID

-- For PhoneTotal
WHERE PP.BusinessEntityID = BEC.BusinessEntityID

```

The first GROUP BY clause is used in the CTE (Common Table Expression) named EmailCount to aggregate the total number of email addresses by BusinessEntityID. The WHERE clause in the subquery for counting phone numbers ensures that the count is specific to each BusinessEntityID in the main query's context.

**How to fix it:**

- To address EmailTotal returning NULL, I can verify if there is a LEFT JOIN between Person.BusinessEntityContact and the EmailCount CTE 

- I can improve PhoneTotal join by testing the data and and checking if there are values that match

- I can also improve my alias names by creating more discriptive names

In [ ]:
USE AdventureWorks2017;
WITH
    EmailCount
    AS
    (
        SELECT
            BusinessEntityID,
            COUNT(EmailAddressID) AS EmailTotal
        FROM Person.EmailAddress
        GROUP BY BusinessEntityID
    )

SELECT
    BEC.BusinessEntityID,
    EC.EmailTotal,
    (SELECT COUNT(PhoneNumber)
    FROM Person.PersonPhone PP
    WHERE PP.BusinessEntityID = BEC.BusinessEntityID) AS PhoneTotal
FROM
    Person.BusinessEntityContact BEC
    LEFT JOIN EmailCount EC ON BEC.BusinessEntityID = EC.BusinessEntityID
GROUP BY BEC.BusinessEntityID, EC.EmailTotal
ORDER BY BEC.BusinessEntityID;


--JSON
USE AdventureWorks2017;
WITH
    EmailCount
    AS
    (
        SELECT
            BusinessEntityID,
            COUNT(EmailAddressID) AS EmailTotal
        FROM Person.EmailAddress
        GROUP BY BusinessEntityID
    )

SELECT
    BEC.BusinessEntityID,
    EC.EmailTotal,
    (SELECT COUNT(PhoneNumber)
    FROM Person.PersonPhone PP
    WHERE PP.BusinessEntityID = BEC.BusinessEntityID) AS PhoneTotal
FROM
    Person.BusinessEntityContact BEC
    LEFT JOIN EmailCount EC ON BEC.BusinessEntityID = EC.BusinessEntityID
GROUP BY BEC.BusinessEntityID, EC.EmailTotal
ORDER BY BEC.BusinessEntityID
FOR JSON PATH, ROOT('BusinessEmailCount');


* * *

**Difficulty:** Medium

**Proposition:** Aggregate total sales per customer by summarizing sales from all their orders.

**Table:** Sales.Customer, Sales.Order, Sales.OrderDetailAudit, Production.Product

**Columns:**

Sales.Customer: CustomerId, CustomerCompanyName Sales.Order: OrderId, CustomerId Sales.OrderDetailAudit: OrderId, ProductId, NewVal (Quantity) Production.Product: ProductId, UnitPrice

**Predicate:**

```
SUM(CONVERT(int, oda.NewVal) * p.UnitPrice) AS TotalSales

```

This calculation in the CTE (Common Table Expression) computes the total sales for each order by multiplying the quantity (NewVal from Sales.OrderDetailAudit, converted to integer for calculation) by the unit price of each product (UnitPrice from Production.Product).

**How to fix it:**

- since Sales.OrderDetailAudit is empty can use a different table like Sales.OrderDetail that has data in the table
    
- I should've checked before I created the query to see if the Sales.OrderDetailAudit table had data
    
- I can also improve my alias names by creating more discriptive names

In [ ]:
USE Northwinds2022TSQLV7;
GO

WITH
    OrderSales
    AS
    (
        SELECT
            o.CustomerId,
            o.OrderId,
            SUM(CONVERT(int, oda.NewVal) * p.UnitPrice) AS TotalSales
        FROM Sales.OrderDetailAudit oda
            INNER JOIN Sales.[Order] o ON oda.OrderId = o.OrderId
            INNER JOIN Production.Product p ON oda.ProductId = p.ProductId
        GROUP BY o.CustomerId, o.OrderId
    )
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    SUM(os.TotalSales) AS TotalSalesAmount
FROM OrderSales os
    INNER JOIN Sales.Customer c ON os.CustomerId = c.CustomerId
GROUP BY c.CustomerId, c.CustomerCompanyName
ORDER BY TotalSalesAmount DESC;


--JSON
USE Northwinds2022TSQLV7;
GO

WITH
    OrderSales
    AS
    (
        SELECT
            o.CustomerId,
            o.OrderId,
            SUM(CONVERT(int, oda.NewVal) * p.UnitPrice) AS TotalSales
        FROM Sales.OrderDetailAudit oda
            INNER JOIN Sales.[Order] o ON oda.OrderId = o.OrderId
            INNER JOIN Production.Product p ON oda.ProductId = p.ProductId
        GROUP BY o.CustomerId, o.OrderId
    )
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    SUM(os.TotalSales) AS TotalSalesAmount
FROM OrderSales os
    INNER JOIN Sales.Customer c ON os.CustomerId = c.CustomerId
GROUP BY c.CustomerId, c.CustomerCompanyName
ORDER BY TotalSalesAmount DESC
FOR JSON PATH, ROOT('TotalNRepairCosts');

***
**Difficulty:** Medium

**Proposition:** Retrieve the average sales amount by currency for orders placed in 2014, sorted by the overall average sales amount in descending order.

**Table:** dbo.FactInternetSales, dbo.DimCurrency, dbo.DimDate

**Columns:** CurrencyName, CurrencyAlternateKey, and OverallAverageSalesAmount calculated from the dbo.FactInternetSales and dbo.DimCurrency tables.

**Predicate:**

```
WHERE f.OrderDateKey IN (SELECT DateKey FROM dbo.DimDate WHERE CalendarYear = 2014)
```

This WHERE clause filters the dbo.FactInternetSales table to only return rows where the OrderDateKey matches any DateKey from the dbo.DimDate table for the year 2014.

**How to fix it:**

- I can remove the calculation of an average of averages (AVG(a.AvgSalesAmount)) because it can lead to misleading results since it does not take into account the variance in the number of transactions per currency

- I can directly calculate the avg in the main query instead of in the cte

- I can also improve my alias names by creating more discriptive names

In [ ]:

USE AdventureWorksDW2017;
GO
WITH
    AverageSalesByCurrency
    AS
    (
        SELECT
            f.CurrencyKey,
            AVG(f.SalesAmount) AS AvgSalesAmount
        FROM
            dbo.FactInternetSales f
            INNER JOIN
            dbo.DimCurrency c ON f.CurrencyKey = c.CurrencyKey
        WHERE
        f.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE CalendarYear = 2014)
        GROUP BY
        f.CurrencyKey
    )
SELECT
    c.CurrencyName,
    c.CurrencyAlternateKey,
    AVG(a.AvgSalesAmount) AS OverallAverageSalesAmount
FROM
    AverageSalesByCurrency a
    INNER JOIN
    dbo.DimCurrency c ON a.CurrencyKey = c.CurrencyKey
GROUP BY
    c.CurrencyName, c.CurrencyAlternateKey
ORDER BY
    OverallAverageSalesAmount DESC;


-- JSON
USE AdventureWorksDW2017;
GO
WITH
    AverageSalesByCurrency
    AS
    (
        SELECT
            f.CurrencyKey,
            AVG(f.SalesAmount) AS AvgSalesAmount
        FROM
            dbo.FactInternetSales f
            INNER JOIN
            dbo.DimCurrency c ON f.CurrencyKey = c.CurrencyKey
        WHERE
        f.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE CalendarYear = 2014)
        GROUP BY
        f.CurrencyKey
    )
SELECT
    c.CurrencyName,
    c.CurrencyAlternateKey,
    AVG(a.AvgSalesAmount) AS OverallAverageSalesAmount
FROM
    AverageSalesByCurrency a
    INNER JOIN
    dbo.DimCurrency c ON a.CurrencyKey = c.CurrencyKey
GROUP BY
    c.CurrencyName, c.CurrencyAlternateKey
ORDER BY
    OverallAverageSalesAmount DESC
FOR JSON PATH, ROOT('AverageSalesAmount');

    ---
   /* SELECT CalendarYear
    FROM dbo.DimDate;*/


# FIXED

In [ ]:
USE AdventureWorks2017;
WITH
    EmailCount
    AS
    (
        SELECT
            BusinessEntityID,
            COUNT(EmailAddressID) AS EmailTotal
        FROM Person.EmailAddress
        GROUP BY BusinessEntityID
    )

SELECT
    BEC.BusinessEntityID,
    EC.EmailTotal,
    (SELECT COUNT(PhoneNumber)
    FROM Person.PersonPhone PersonPhoneNumber
    WHERE PersonPhoneNumber.BusinessEntityID = BEC.BusinessEntityID) AS PhoneTotal
FROM
    Person.BusinessEntityContact BEC
    LEFT JOIN EmailCount EC ON BEC.BusinessEntityID = EC.BusinessEntityID
GROUP BY BEC.BusinessEntityID, EC.EmailTotal
ORDER BY BEC.BusinessEntityID;


--JSON
USE AdventureWorks2017;
WITH
    EmailCount
    AS
    (
        SELECT
            BusinessEntityID,
            COUNT(EmailAddressID) AS EmailTotal
        FROM Person.EmailAddress
        GROUP BY BusinessEntityID
    )

SELECT
    BEC.BusinessEntityID,
    EC.EmailTotal,
    (SELECT COUNT(PhoneNumber)
    FROM Person.PersonPhone PersonPhoneNumber
    WHERE PersonPhoneNumber.BusinessEntityID = BEC.BusinessEntityID) AS PhoneTotal
FROM
    Person.BusinessEntityContact BEC
    LEFT JOIN EmailCount EC ON BEC.BusinessEntityID = EC.BusinessEntityID
GROUP BY BEC.BusinessEntityID, EC.EmailTotal
ORDER BY BEC.BusinessEntityID
FOR JSON PATH, ROOT('BusinessEmailCount');


In [ ]:
USE Northwinds2022TSQLV7;
GO

WITH
    OrderSales
    AS
    (
        SELECT
            OrderD.CustomerId,
            OrderD.OrderId,
            SUM(CONVERT(int, OrderDA.NewVal) * p.UnitPrice) AS TotalSales
        FROM Sales.OrderDetailAudit OrderDA
            INNER JOIN Sales.[Order] OrderD ON OrderDA.OrderId = OrderD.OrderId
            INNER JOIN Production.Product p ON OrderDA.ProductId = p.ProductId
        GROUP BY OrderD.CustomerId, OrderD.OrderId
    )
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    SUM(os.TotalSales) AS TotalSalesAmount
FROM OrderSales os
    INNER JOIN Sales.Customer c ON os.CustomerId = c.CustomerId
GROUP BY c.CustomerId, c.CustomerCompanyName
ORDER BY TotalSalesAmount DESC;


--JSON
USE Northwinds2022TSQLV7;
GO

WITH
    OrderSales
    AS
    (
        SELECT
            OrderD.CustomerId,
            OrderD.OrderId,
            SUM(CONVERT(int, OrderDA.NewVal) * p.UnitPrice) AS TotalSales
        FROM Sales.OrderDetailAudit OrderDA
            INNER JOIN Sales.[Order] OrderD ON OrderDA.OrderId = OrderD.OrderId
            INNER JOIN Production.Product p ON OrderDA.ProductId = p.ProductId
        GROUP BY OrderD.CustomerId, OrderD.OrderId
    )
SELECT
    c.CustomerId,
    c.CustomerCompanyName,
    SUM(os.TotalSales) AS TotalSalesAmount
FROM OrderSales os
    INNER JOIN Sales.Customer c ON os.CustomerId = c.CustomerId
GROUP BY c.CustomerId, c.CustomerCompanyName
ORDER BY TotalSalesAmount DESC
FOR JSON PATH, ROOT('TotalNRepairCosts');

In [ ]:

USE AdventureWorksDW2017;
GO
WITH
    AverageSalesByCurrency
    AS
    (
        SELECT
            f.CurrencyKey,
            AVG(f.SalesAmount) AS AvgSalesAmount
        FROM
            dbo.FactInternetSales f
            INNER JOIN
            dbo.DimCurrency c ON f.CurrencyKey = c.CurrencyKey
        WHERE
        f.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE CalendarYear = 2014)
        GROUP BY
        f.CurrencyKey
    )
SELECT
    c.CurrencyName,
    c.CurrencyAlternateKey,
    AVG(a.AvgSalesAmount) AS OverallAverageSalesAmount
FROM
    AverageSalesByCurrency a
    INNER JOIN
    dbo.DimCurrency c ON a.CurrencyKey = c.CurrencyKey
GROUP BY
    c.CurrencyName, c.CurrencyAlternateKey
ORDER BY
    OverallAverageSalesAmount DESC;


-- JSON
USE AdventureWorksDW2017;
GO
WITH
    AverageSalesByCurrency
    AS
    (
        SELECT
            f.CurrencyKey,
            AVG(f.SalesAmount) AS AvgSalesAmount
        FROM
            dbo.FactInternetSales f
            INNER JOIN
            dbo.DimCurrency c ON f.CurrencyKey = c.CurrencyKey
        WHERE
        f.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE CalendarYear = 2014)
        GROUP BY
        f.CurrencyKey
    )
SELECT
    c.CurrencyName,
    c.CurrencyAlternateKey,
    AVG(a.AvgSalesAmount) AS OverallAverageSalesAmount
FROM
    AverageSalesByCurrency a
    INNER JOIN
    dbo.DimCurrency c ON a.CurrencyKey = c.CurrencyKey
GROUP BY
    c.CurrencyName, c.CurrencyAlternateKey
ORDER BY
    OverallAverageSalesAmount DESC
FOR JSON PATH, ROOT('AverageSalesAmount');

    ---
   /* SELECT CalendarYear
    FROM dbo.DimDate;*/


# REST OF MY MEDIUM QUERIES

* * *

**Difficulty:** Medium 1

**Proposition:** Retrieve sales employee details with total orders since hiring after January 1, 2012, ordered by total orders in descending order.

**Table:** HumanResources.Employee, Sales.Order

**Columns:** EmployeeFirstName, EmployeeLastName, EmployeeRegion, and TotalOrders from a combined dataset of HumanResources.Employee and a derived table EmployeeSales that aggregates Order counts by EmployeeId.

**Predicate:**

```
WHERE HireDate > '2012-01-01'

```

This WHERE clause filters the HumanResources.Employee table to only return employees hired after January 1, 2012.

In [ ]:

USE Northwinds2022TSQLV7;

WITH
    EmployeeSales
    AS
    (
        SELECT
            o.EmployeeId,
            COUNT(o.OrderId) AS TotalOrders
        FROM
            Sales.[Order] o
        GROUP BY
        o.EmployeeId
    )

SELECT
    e.EmployeeFirstName,
    e.EmployeeLastName,
    e.EmployeeRegion,
    es.TotalOrders
FROM
    HumanResources.Employee e
    INNER JOIN
    EmployeeSales es ON e.EmployeeId = es.EmployeeId
WHERE
    e.EmployeeId IN (SELECT EmployeeId
FROM HumanResources.Employee
WHERE HireDate > '2012-01-01')
GROUP BY
    e.EmployeeFirstName, e.EmployeeLastName, e.EmployeeRegion, es.TotalOrders
ORDER BY
    es.TotalOrders DESC;


-- JSON
USE Northwinds2022TSQLV7;

WITH
    EmployeeSales
    AS
    (
        SELECT
            o.EmployeeId,
            COUNT(o.OrderId) AS TotalOrders
        FROM
            Sales.[Order] o
        GROUP BY
        o.EmployeeId
    )

SELECT
    e.EmployeeFirstName,
    e.EmployeeLastName,
    e.EmployeeRegion,
    es.TotalOrders
FROM
    HumanResources.Employee e
    INNER JOIN
    EmployeeSales es ON e.EmployeeId = es.EmployeeId
WHERE
    e.EmployeeId IN (SELECT EmployeeId
FROM HumanResources.Employee
WHERE HireDate > '2012-01-01')
GROUP BY
    e.EmployeeFirstName, e.EmployeeLastName, e.EmployeeRegion, es.TotalOrders
ORDER BY
    es.TotalOrders DESC
FOR JSON PATH, ROOT('SalesEmployeeDetails');


* * *

**Difficulty:** Medium 2

**Proposition:** Retrieve average pay rates per department, including the number of unique positions in each, ordered by average pay rate in descending order.

**Table:** HumanResources.EmployeePayHistory, HumanResources.EmployeeDepartmentHistory, HumanResources.Department

**Columns:** DepartmentName, AveragePayRate, and NumberOfUniquePositions from a combination of HumanResources.EmployeePayHistory, HumanResources.EmployeeDepartmentHistory, and HumanResources.Department tables.

**Predicate:**

```
WHERE edh.EndDate IS NULL

```

This WHERE clause filters the EmployeeDepartmentHistory to only consider current department positions, i.e., those without an EndDate.

```
WHERE edh.EndDate IS NULL AND edh.DepartmentID = dps.DepartmentID

```

used within a subquery to match the DepartmentID from the EmployeeDepartmentHistory to the DepartmentID in the DepartmentPaySummary CTE, ensuring that the count of unique positions is accurate for each department.

In [ ]:

USE AdventureWorks2017;

WITH
    DepartmentPaySummary
    AS
    (
        SELECT
            edh.DepartmentID,
            AVG(eph.Rate) AS AvgRate
        FROM HumanResources.EmployeePayHistory eph
            INNER JOIN HumanResources.EmployeeDepartmentHistory edh
            ON eph.BusinessEntityID = edh.BusinessEntityID
        WHERE edh.EndDate IS NULL
        GROUP BY edh.DepartmentID
    )

SELECT
    d.Name AS DepartmentName,
    dps.AvgRate AS AveragePayRate,
    (SELECT COUNT(DISTINCT edh.BusinessEntityID)
    FROM HumanResources.EmployeeDepartmentHistory edh
    WHERE edh.DepartmentID = dps.DepartmentID
        AND edh.EndDate IS NULL) AS NumberOfUniquePositions
FROM DepartmentPaySummary dps
    INNER JOIN HumanResources.Department d
    ON dps.DepartmentID = d.DepartmentID
ORDER BY AveragePayRate DESC;



--JSON
USE AdventureWorks2017;

WITH
    DepartmentPaySummary
    AS
    (
        SELECT
            edh.DepartmentID,
            AVG(eph.Rate) AS AvgRate
        FROM HumanResources.EmployeePayHistory eph
            INNER JOIN HumanResources.EmployeeDepartmentHistory edh
            ON eph.BusinessEntityID = edh.BusinessEntityID
        WHERE edh.EndDate IS NULL
        GROUP BY edh.DepartmentID
    )

SELECT
    d.Name AS DepartmentName,
    dps.AvgRate AS AveragePayRate,
    (SELECT COUNT(DISTINCT edh.BusinessEntityID)
    FROM HumanResources.EmployeeDepartmentHistory edh
    WHERE edh.DepartmentID = dps.DepartmentID
        AND edh.EndDate IS NULL) AS NumberOfUniquePositions
FROM DepartmentPaySummary dps
    INNER JOIN HumanResources.Department d
    ON dps.DepartmentID = d.DepartmentID
ORDER BY AveragePayRate DESC
FOR JSON PATH, ROOT('AveragePayRatesPerDepartment');



* * *

**Difficulty:** Medium 3

**Proposition:** <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);"> Write a query that calculates and retrieves the total sales by each employee in a specific fiscal year, ordered by the total sales in descending order.</span>

**Table:** dbo.FactResellerSales, dbo.DimEmployee, dbo.DimSalesTerritory

**Columns:** EmployeeKey, FirstName, LastName, SalesTerritoryRegion, and the calculated TotalSales from the dbo.FactResellerSales table.

**Predicate:**

```
WHERE frs.OrderDateKey IN (SELECT DateKey FROM dbo.DimDate WHERE FiscalYear = 2013)

```

This WHERE clause filters the dbo.FactResellerSales table to only return rows where the OrderDateKey matches any DateKey from the dbo.DimDate table that falls in the fiscal year 2013.

In [ ]:
USE AdventureWorksDW2017;
WITH
    EmployeeSales
    AS
    (
        SELECT
            e.EmployeeKey,
            e.FirstName,
            e.LastName,
            st.SalesTerritoryRegion,
            SUM(frs.SalesAmount) AS TotalSales
        FROM
            dbo.FactResellerSales frs
            INNER JOIN
            dbo.DimEmployee e ON frs.EmployeeKey = e.EmployeeKey
            INNER JOIN
            dbo.DimSalesTerritory st ON e.SalesTerritoryKey = st.SalesTerritoryKey
        WHERE
        frs.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE FiscalYear = 2013)
        GROUP BY
        e.EmployeeKey, e.FirstName, e.LastName, st.SalesTerritoryRegion
    )

SELECT
    es.FirstName,
    es.LastName,
    es.SalesTerritoryRegion,
    SUM(es.TotalSales) AS TotalSalesByEmployee
FROM
    EmployeeSales es
GROUP BY
    es.FirstName, es.LastName, es.SalesTerritoryRegion
ORDER BY
    TotalSalesByEmployee DESC;



--JSON
USE AdventureWorksDW2017;
WITH
    EmployeeSales
    AS
    (
        SELECT
            e.EmployeeKey,
            e.FirstName,
            e.LastName,
            st.SalesTerritoryRegion,
            SUM(frs.SalesAmount) AS TotalSales
        FROM
            dbo.FactResellerSales frs
            INNER JOIN
            dbo.DimEmployee e ON frs.EmployeeKey = e.EmployeeKey
            INNER JOIN
            dbo.DimSalesTerritory st ON e.SalesTerritoryKey = st.SalesTerritoryKey
        WHERE
        frs.OrderDateKey IN (SELECT DateKey
        FROM dbo.DimDate
        WHERE FiscalYear = 2013)
        GROUP BY
        e.EmployeeKey, e.FirstName, e.LastName, st.SalesTerritoryRegion
    )

SELECT
    es.FirstName,
    es.LastName,
    es.SalesTerritoryRegion,
    SUM(es.TotalSales) AS TotalSalesByEmployee
FROM
    EmployeeSales es
GROUP BY
    es.FirstName, es.LastName, es.SalesTerritoryRegion
ORDER BY
    TotalSalesByEmployee DESC
FOR JSON PATH, ROOT('TotalSalesEachEmployee');



* * *

**Difficulty:** Medium 4

**Proposition:** Retrieve detailed summary of purchase orders, including the latest receipt date for each order.

**Table:** Purchasing.PurchaseOrderLines, Purchasing.PurchaseOrders

**Columns:** PurchaseOrderID, SupplierID, OrderDate, TotalItemsOrdered, TotalExpectedCost, and LatestReceiptDate from the Purchasing.PurchaseOrderLines and Purchasing.PurchaseOrders tables.

**Predicate:**

```
WHERE pol2.PurchaseOrderID = pol.PurchaseOrderID

```

This WHERE clause is used in a subquery to ensure that the MAX function applies to the LastReceiptDate of each specific PurchaseOrderID in the Purchasing.PurchaseOrderLines table. It filters the rows to only return the latest LastReceiptDate for each PurchaseOrderID.

In [ ]:

USE WideWorldimporters;
GO
WITH
    PurchaseOrderSummary
    AS
    (
        SELECT
            pol.PurchaseOrderID,
            COUNT(pol.PurchaseOrderLineID) AS TotalItemsOrdered,
            SUM(pol.OrderedOuters * pol.ExpectedUnitPricePerOuter) AS TotalExpectedCost,
            (SELECT MAX(pol2.LastReceiptDate)
            FROM Purchasing.PurchaseOrderLines pol2
            WHERE pol2.PurchaseOrderID = pol.PurchaseOrderID) AS LatestReceiptDate
        FROM
            Purchasing.PurchaseOrderLines pol
        GROUP BY
        pol.PurchaseOrderID
    )
SELECT
    pos.PurchaseOrderID,
    po.SupplierID,
    po.OrderDate,
    pos.TotalItemsOrdered,
    pos.TotalExpectedCost,
    pos.LatestReceiptDate
FROM
    PurchaseOrderSummary pos
    INNER JOIN
    Purchasing.PurchaseOrders po ON pos.PurchaseOrderID = po.PurchaseOrderID
ORDER BY
    po.OrderDate DESC;


--JSON
USE WideWorldimporters;
GO
WITH
    PurchaseOrderSummary
    AS
    (
        SELECT
            pol.PurchaseOrderID,
            COUNT(pol.PurchaseOrderLineID) AS TotalItemsOrdered,
            SUM(pol.OrderedOuters * pol.ExpectedUnitPricePerOuter) AS TotalExpectedCost,
            (SELECT MAX(pol2.LastReceiptDate)
            FROM Purchasing.PurchaseOrderLines pol2
            WHERE pol2.PurchaseOrderID = pol.PurchaseOrderID) AS LatestReceiptDate
        FROM
            Purchasing.PurchaseOrderLines pol
        GROUP BY
        pol.PurchaseOrderID
    )
SELECT
    pos.PurchaseOrderID,
    po.SupplierID,
    po.OrderDate,
    pos.TotalItemsOrdered,
    pos.TotalExpectedCost,
    pos.LatestReceiptDate
FROM
    PurchaseOrderSummary pos
    INNER JOIN
    Purchasing.PurchaseOrders po ON pos.PurchaseOrderID = po.PurchaseOrderID
ORDER BY
    po.OrderDate DESC
FOR JSON PATH, ROOT('DetailedPurchaseOrders');




* * *

**Difficulty:** Medium 5

Proposition: Retrieve the total sales for each product sold in a specific sales territory during the year 2014, ordered by total sales in descending order.

Table: dbo.FactInternetSales, dbo.DimProduct

Columns: EnglishProductName from dbo.DimProduct and TotalSales from a derived table ProductSales.

Predicate:

```
WHERE f.OrderDateKey IN (SELECT DateKey FROM dbo.DimDate WHERE YEAR(FullDateAlternateKey) = 2014)
AND f.SalesTerritoryKey = 1

```

This WHERE clause filters the dbo.FactInternetSales table to only return rows where the OrderDateKey matches any DateKey from the dbo.DimDate table for the year 2014 and where the SalesTerritoryKey equals 1.

In [ ]:
USE AdventureWorksDW2017;
GO

WITH
    ProductSales
    AS
    (
        SELECT
            f.ProductKey,
            SUM(f.SalesAmount) AS TotalSales
        FROM
            dbo.FactInternetSales f
        WHERE
        f.OrderDateKey IN (SELECT DateKey
            FROM dbo.DimDate
            WHERE YEAR(FullDateAlternateKey) = 2014)
            AND f.SalesTerritoryKey = 1
        GROUP BY
        f.ProductKey
    )
SELECT
    p.EnglishProductName,
    ps.TotalSales
FROM
    ProductSales ps
    INNER JOIN
    dbo.DimProduct p ON ps.ProductKey = p.ProductKey
ORDER BY
    ps.TotalSales DESC;


--JSON
USE AdventureWorksDW2017;
GO

WITH
    ProductSales
    AS
    (
        SELECT
            f.ProductKey,
            SUM(f.SalesAmount) AS TotalSales
        FROM
            dbo.FactInternetSales f
        WHERE
        f.OrderDateKey IN (SELECT DateKey
            FROM dbo.DimDate
            WHERE YEAR(FullDateAlternateKey) = 2014)
            AND f.SalesTerritoryKey = 1
        GROUP BY
        f.ProductKey
    )
SELECT
    p.EnglishProductName,
    ps.TotalSales
FROM
    ProductSales ps
    INNER JOIN
    dbo.DimProduct p ON ps.ProductKey = p.ProductKey
ORDER BY
    ps.TotalSales DESC
FOR JSON PATH, ROOT('TotalSalesPerProduct');

    ---
    SELECT SalesTerritoryKey
    FROM dbo.DimSalesTerritory



* * *

**Difficulty:** Medium 6

**Proposition:** Aggregate and report on the total quantities on hand for each purchase order, sorted by these totals in descending order.

**Table:** Purchasing.PurchaseOrders, Purchasing.PurchaseOrderLines, Warehouse.StockItemHoldings

**Columns:** PurchaseOrderID, OrderDate from Purchasing.PurchaseOrders, StockItemID, QuantityOnHand from Warehouse.StockItemHoldings.

**Predicate:**

```
(SELECT SUM(ss.TotalQuantityOnHand)
 FROM StockSummary ss
 INNER JOIN Purchasing.PurchaseOrderLines pol ON ss.StockItemID = pol.StockItemID
 WHERE pol.PurchaseOrderID = po.PurchaseOrderID) AS TotalQuantityOnHandForOrder

```

This subquery calculates the total quantity on hand for the items included in each purchase order by joining the StockSummary CTE (which aggregates QuantityOnHand by StockItemID) with the PurchaseOrderLines table. It filters based on the PurchaseOrderID to ensure that the quantities on hand are accurately summed for items relevant to each purchase order.

In [ ]:
USE WideWorldimporters;
GO
WITH
    StockSummary
    AS
    (
        SELECT
            StockItemID,
            SUM(QuantityOnHand) AS TotalQuantityOnHand
        FROM
            Warehouse.StockItemHoldings
        GROUP BY
        StockItemID
    )
SELECT
    po.PurchaseOrderID,
    po.OrderDate,
    (SELECT SUM(ss.TotalQuantityOnHand)
    FROM StockSummary ss
        INNER JOIN Purchasing.PurchaseOrderLines pol ON ss.StockItemID = pol.StockItemID
    WHERE pol.PurchaseOrderID = po.PurchaseOrderID) AS TotalQuantityOnHandForOrder
FROM
    Purchasing.PurchaseOrders po
GROUP BY
    po.PurchaseOrderID,
    po.OrderDate
ORDER BY
    TotalQuantityOnHandForOrder DESC;


--JSON
USE WideWorldimporters;
GO
WITH
    StockSummary
    AS
    (
        SELECT
            StockItemID,
            SUM(QuantityOnHand) AS TotalQuantityOnHand
        FROM
            Warehouse.StockItemHoldings
        GROUP BY
        StockItemID
    )
SELECT
    po.PurchaseOrderID,
    po.OrderDate,
    (SELECT SUM(ss.TotalQuantityOnHand)
    FROM StockSummary ss
        INNER JOIN Purchasing.PurchaseOrderLines pol ON ss.StockItemID = pol.StockItemID
    WHERE pol.PurchaseOrderID = po.PurchaseOrderID) AS TotalQuantityOnHandForOrder
FROM
    Purchasing.PurchaseOrders po
GROUP BY
    po.PurchaseOrderID,
    po.OrderDate
ORDER BY
    TotalQuantityOnHandForOrder DESC
FOR JSON PATH, ROOT('TotalNumberQuantities');



* * *

**Difficulty:** Medium 7

**Proposition:** Analyze sales performance of stock items, focusing on total quantity sold and the most recent sale amount for each item.

**Table:** Fact.Sale, Dimension.\[Stock Item\]

**Columns:** \[Stock Item\] from Dimension.\[Stock Item\], TotalQuantitySold from an aggregated calculation, and MostRecentSaleAmount as a derived value from Fact.Sale.

**Predicate:**

```
WHERE [Stock Item Key] = SI.[Stock Item Key]

```

This WHERE clause is used within a subquery to ensure that the most recent sale amount is accurately fetched for the corresponding stock item by matching the \[Stock Item Key\] between the Fact.Sale table and the Dimension.\[Stock Item\] table.

In [ ]:
USE WideWorldImportersDW;
GO

WITH
    StockItemSales
    AS
    (
        SELECT
            [Stock Item Key],
            SUM(Quantity) AS TotalQuantitySold
        FROM
            Fact.Sale
        GROUP BY
        [Stock Item Key]
    )
SELECT
    SI.[Stock Item],
    SIS.TotalQuantitySold,
    (
        SELECT TOP 1
        [Total Including Tax]
    FROM Fact.Sale
    WHERE [Stock Item Key] = SI.[Stock Item Key]
    ORDER BY [Invoice Date Key] DESC
    ) AS MostRecentSaleAmount
FROM
    Dimension.[Stock Item] SI
    INNER JOIN StockItemSales SIS ON SI.[Stock Item Key] = SIS.[Stock Item Key]
ORDER BY
    SIS.TotalQuantitySold DESC;



--JSON
USE WideWorldImportersDW;
GO

WITH
    StockItemSales
    AS
    (
        SELECT
            [Stock Item Key],
            SUM(Quantity) AS TotalQuantitySold
        FROM
            Fact.Sale
        GROUP BY
        [Stock Item Key]
    )
SELECT
    SI.[Stock Item],
    SIS.TotalQuantitySold,
    (
        SELECT TOP 1
        [Total Including Tax]
    FROM Fact.Sale
    WHERE [Stock Item Key] = SI.[Stock Item Key]
    ORDER BY [Invoice Date Key] DESC
    ) AS MostRecentSaleAmount
FROM
    Dimension.[Stock Item] SI
    INNER JOIN StockItemSales SIS ON SI.[Stock Item Key] = SIS.[Stock Item Key]
ORDER BY
    SIS.TotalQuantitySold DESC

FOR JSON PATH, ROOT('SalesPerformanceOfStockItems');




* * *

**Difficulty:** Medium 8

**Proposition:** Analyze the relationship between orders and movements by supplier and package, aggregating quantities and movement counts.

**Table:** Fact.\[Order\], Fact.Movement

**Columns:** Description, Package, OrderQuantity from Fact.\[Order\] table, and Supplier Key from both Fact.\[Order\] and Fact.Movement tables. TotalMovements is a calculated column representing the total number of movements per supplier matching the order quantity.

**Predicate:**

```
WHERE mv.[Supplier Key] = m.[Supplier Key] AND mv.Quantity = o.Quantity

```

This WHERE clause is used within a subquery to ensure that the count of movements is specific to each supplier and matches the order quantity. It filters the Fact.Movement table to only include rows where the Supplier Key matches and the movement quantity is equal to the order quantity, thus accurately reflecting the total movements associated with the supplier for each order quantity.

In [ ]:
USE WideWorldImportersDW;

WITH
    OrderSummary
    AS
    (
        SELECT
            o.[Description],
            o.Package,
            o.Quantity AS OrderQuantity,
            m.[Supplier Key],
            (SELECT COUNT(*)
            FROM Fact.Movement mv
            WHERE mv.[Supplier Key] = m.[Supplier Key] AND mv.Quantity = o.Quantity) AS TotalMovements
        FROM
            Fact.[Order] o
            LEFT JOIN Fact.Movement m ON o.Quantity = m.Quantity
    )

SELECT
    os.Package,
    os.[Supplier Key],
    COUNT(*) AS TotalOrders,
    SUM(os.OrderQuantity) AS TotalQuantityOrdered,
    SUM(os.TotalMovements) AS TotalMovements
FROM
    OrderSummary os
GROUP BY
    os.Package,
    os.[Supplier Key]
ORDER BY
    os.Package,
    os.[Supplier Key];


--JSON
USE WideWorldImportersDW;

WITH
    OrderSummary
    AS
    (
        SELECT
            o.[Description],
            o.Package,
            o.Quantity AS OrderQuantity,
            m.[Supplier Key],
            (SELECT COUNT(*)
            FROM Fact.Movement mv
            WHERE mv.[Supplier Key] = m.[Supplier Key] AND mv.Quantity = o.Quantity) AS TotalMovements
        FROM
            Fact.[Order] o
            LEFT JOIN Fact.Movement m ON o.Quantity = m.Quantity
    )

SELECT
    os.Package,
    os.[Supplier Key],
    COUNT(*) AS TotalOrders,
    SUM(os.OrderQuantity) AS TotalQuantityOrdered,
    SUM(os.TotalMovements) AS TotalMovements
FROM
    OrderSummary os
GROUP BY
    os.Package,
    os.[Supplier Key]
ORDER BY
    os.Package,
    os.[Supplier Key]

FOR JSON PATH, ROOT('RelationshipBetweenOrders');




# REST OF MY COMPLEX QUERIES

* * *

**Difficulty:** Complex 1

**Proposition:** Adjust total sales for tax based on the country using a user-defined function for tax adjustment, ensuring that sales data is accurately matched with the correct country information.

**Function:** dbo.AdjustSalesForTax

**Parameters:** @SalePrice as DECIMAL(18, 2) (the total sale price before tax), @CountryISO2 as CHAR(2) (the 2-letter ISO code for the country).

**Returns:** The adjusted sale price as DECIMAL(18, 2), incorporating the tax rate based on the country specified.

**Tables and Columns:**

Data.Sales: TotalSalePrice, CustomerID

Data.Customer: CustomerID, Country

Data.Country: CountryISO2, CountryISO3, CountryName

**Predicate:**

```
IF @CountryISO2 = 'USA'
    SET @TaxRate = 1.20 -- 20% tax
ELSE
    SET @TaxRate = 1.15 -- 15% tax

```

This IF statement adjusts the tax rate depending on whether the country is the USA or another country, allowing for different tax rates to be applied to the sale price.

```
WHERE c.Country = cn.CountryISO2 

```

This WHERE clause ensures sales data is matched with the correct country by comparing the customer's country in Data.Customer with the country ISO code in Data.Country. It's essential for accurately applying tax adjustments based on specific country tax rates.

In [ ]:
USE PrestigeCars ;
GO
CREATE OR ALTER FUNCTION dbo.AdjustSalesForTax (@SalePrice DECIMAL(18, 2), @CountryISO2 CHAR(2))
RETURNS DECIMAL(18, 2)
AS
BEGIN
    DECLARE @TaxRate DECIMAL(5, 2)
    IF @CountryISO2 = 'USA'
        SET @TaxRate = 1.20 -- 20% tax
    ELSE
        SET @TaxRate = 1.15
    -- 15% tax

    RETURN @SalePrice * @TaxRate
END;

GO
WITH
    CountrySalesAdjusted
    AS
    (
        SELECT
            c.Country,
            dbo.AdjustSalesForTax(SUM(s.TotalSalePrice), cn.CountryISO2) AS AdjustedTotalSales
        FROM Data.Sales s
            INNER JOIN Data.Customer c ON s.CustomerID = c.CustomerID
            INNER JOIN Data.Country cn ON c.Country = cn.CountryISO2
        GROUP BY c.Country, cn.CountryISO2
    )

SELECT
    cn.CountryName,
    cn.CountryISO3,
    CSA.AdjustedTotalSales,
    (SELECT AVG(TotalSalePrice)
    FROM Data.Sales s
        INNER JOIN Data.Customer c ON s.CustomerID = c.CustomerID
    WHERE c.Country = cn.CountryISO2) AS AvgSalePerCustomer
FROM CountrySalesAdjusted CSA
    INNER JOIN Data.Country cn ON CSA.Country = cn.CountryISO2
ORDER BY CSA.AdjustedTotalSales DESC;



--JSON
USE PrestigeCars ;
GO
CREATE OR ALTER FUNCTION dbo.AdjustSalesForTax (@SalePrice DECIMAL(18, 2), @CountryISO2 CHAR(2))
RETURNS DECIMAL(18, 2)
AS
BEGIN
    DECLARE @TaxRate DECIMAL(5, 2)
    IF @CountryISO2 = 'USA'
        SET @TaxRate = 1.20 -- 20% tax
    ELSE
        SET @TaxRate = 1.15
    -- 15% tax

    RETURN @SalePrice * @TaxRate
END;

GO
WITH
    CountrySalesAdjusted
    AS
    (
        SELECT
            c.Country,
            dbo.AdjustSalesForTax(SUM(s.TotalSalePrice), cn.CountryISO2) AS AdjustedTotalSales
        FROM Data.Sales s
            INNER JOIN Data.Customer c ON s.CustomerID = c.CustomerID
            INNER JOIN Data.Country cn ON c.Country = cn.CountryISO2
        GROUP BY c.Country, cn.CountryISO2
    )

SELECT
    cn.CountryName,
    cn.CountryISO3,
    CSA.AdjustedTotalSales,
    (SELECT AVG(TotalSalePrice)
    FROM Data.Sales s
        INNER JOIN Data.Customer c ON s.CustomerID = c.CustomerID
    WHERE c.Country = cn.CountryISO2) AS AvgSalePerCustomer
FROM CountrySalesAdjusted CSA
    INNER JOIN Data.Country cn ON CSA.Country = cn.CountryISO2
ORDER BY CSA.AdjustedTotalSales DESC
FOR JSON PATH, ROOT('CountrySalesTax');





* * *

**Difficulty:** Complex 2

**Proposition:** Analyze sales and customer ratings across product categories, utilizing a user-defined function for average rating calculation.

**Function:** CalculateAverageRating

**Parameters:**

@ProductID as INT (identifies the product). Returns: Average rating as FLOAT, calculated from customer reviews for the specified product.

**Tables and Columns:**

Production.Product: ProductID, Name, ProductSubcategoryID Sales.SalesOrderDetail: ProductID, OrderQty, UnitPrice Production.ProductReview: ProductID, Rating Production.ProductSubcategory: ProductSubcategoryID, ProductCategoryID Production.ProductCategory: ProductCategoryID, Name

**Predicate:**

```
WHERE ProductID = @ProductID

```

This predicate is used within the CalculateAverageRating function to filter the Production.ProductReview table for reviews of the specified product.

```
WHERE ProductCategoryID = PC.ProductCategoryID

```

This predicate is added to the subquery within the main query to ensure that the count of distinct products is limited to those within the specific product category being analyzed.

In [ ]:
USE AdventureWorks2017;
GO
CREATE OR ALTER FUNCTION CalculateAverageRating(@ProductID INT)
RETURNS FLOAT
AS
BEGIN
    RETURN (SELECT AVG(CAST(Rating AS FLOAT))
    FROM Production.ProductReview
    WHERE ProductID = @ProductID)
END

GO

WITH
    SalesAndRatings
    AS
    (
        SELECT
            P.ProductID,
            P.Name,
            PSC.ProductSubcategoryID,
            PSC.ProductCategoryID,
            SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales,
            dbo.CalculateAverageRating(P.ProductID) AS AverageRating
        FROM Sales.SalesOrderDetail SOD
            INNER JOIN Production.Product P ON SOD.ProductID = P.ProductID
            LEFT JOIN Production.ProductSubcategory PSC ON P.ProductSubcategoryID = PSC.ProductSubcategoryID
        GROUP BY P.ProductID, P.Name, PSC.ProductSubcategoryID, PSC.ProductCategoryID
    )

SELECT
    PC.Name AS CategoryName,
    AVG(SAR.TotalSales) AS AvgCategorySales,
    AVG(SAR.AverageRating) AS AvgCategoryRating,
    (SELECT COUNT(DISTINCT ProductID)
    FROM Production.Product P
    WHERE P.ProductSubcategoryID IN (
         SELECT ProductSubcategoryID
    FROM Production.ProductSubcategory
    WHERE ProductCategoryID = PC.ProductCategoryID
     )) AS ProductsCount
FROM SalesAndRatings SAR
    INNER JOIN Production.ProductCategory PC ON SAR.ProductCategoryID = PC.ProductCategoryID
GROUP BY PC.Name, PC.ProductCategoryID
ORDER BY AvgCategorySales DESC;



--JSON
USE AdventureWorks2017;
GO
CREATE OR ALTER FUNCTION CalculateAverageRating(@ProductID INT)
RETURNS FLOAT
AS
BEGIN
    RETURN (SELECT AVG(CAST(Rating AS FLOAT))
    FROM Production.ProductReview
    WHERE ProductID = @ProductID)
END

GO

WITH
    SalesAndRatings
    AS
    (
        SELECT
            P.ProductID,
            P.Name,
            PSC.ProductSubcategoryID,
            PSC.ProductCategoryID,
            SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales,
            dbo.CalculateAverageRating(P.ProductID) AS AverageRating
        FROM Sales.SalesOrderDetail SOD
            INNER JOIN Production.Product P ON SOD.ProductID = P.ProductID
            LEFT JOIN Production.ProductSubcategory PSC ON P.ProductSubcategoryID = PSC.ProductSubcategoryID
        GROUP BY P.ProductID, P.Name, PSC.ProductSubcategoryID, PSC.ProductCategoryID
    )

SELECT
    PC.Name AS CategoryName,
    AVG(SAR.TotalSales) AS AvgCategorySales,
    AVG(SAR.AverageRating) AS AvgCategoryRating,
    (SELECT COUNT(DISTINCT ProductID)
    FROM Production.Product P
    WHERE P.ProductSubcategoryID IN (
         SELECT ProductSubcategoryID
    FROM Production.ProductSubcategory
    WHERE ProductCategoryID = PC.ProductCategoryID
     )) AS ProductsCount
FROM SalesAndRatings SAR
    INNER JOIN Production.ProductCategory PC ON SAR.ProductCategoryID = PC.ProductCategoryID
GROUP BY PC.Name, PC.ProductCategoryID
ORDER BY AvgCategorySales DESC
FOR JSON PATH, ROOT('SalesAndCustomerRatings');



* * *

**Difficulty:** Complex 3

**Proposition:** Categorize sales performance per product within sales territories using a user-defined function to determine sales categories based on total sales.

**Function:** dbo.CalculateSalesCategory

**Parameters:** @SalesAmount as MONEY (total sales amount for the product).

**Returns:** A NVARCHAR(50) string indicating the sales category ('Low', 'Medium', or 'High') based on predefined sales amount thresholds.

**Tables and Columns:**

dbo.FactResellerSales: ProductKey, SalesAmount, SalesTerritoryKey dbo.DimProduct: ProductKey, EnglishProductName dbo.DimSalesTerritory: SalesTerritoryKey, SalesTerritoryRegion

**Predicate:**

```
WHERE ss.TotalSales > (SELECT AVG(TotalSales) * 1.5 FROM SalesSummary)

```

This WHERE clause filters the aggregated sales data to only include products whose total sales exceed 150% of the average total sales across all summarized entries, focusing analysis on higher-performing products.

In [ ]:
USE AdventureWorksDW2017;
GO
CREATE OR ALTER FUNCTION dbo.CalculateSalesCategory(@SalesAmount MONEY)
RETURNS NVARCHAR(50)
AS
BEGIN
    RETURN CASE
        WHEN @SalesAmount < 50000 THEN 'Low'
        WHEN @SalesAmount BETWEEN 50000 AND 100000 THEN 'Medium'
        ELSE 'High'
    END;
END;

GO

WITH
    SalesSummary
    AS
    (
        SELECT
            p.ProductKey,
            p.EnglishProductName,
            rs.SalesTerritoryKey,
            SUM(rs.SalesAmount) AS TotalSales
        FROM
            dbo.FactResellerSales rs
            INNER JOIN
            dbo.DimProduct p ON rs.ProductKey = p.ProductKey
        GROUP BY
        p.ProductKey, p.EnglishProductName, rs.SalesTerritoryKey
    )
SELECT
    sst.SalesTerritoryRegion,
    ss.EnglishProductName,
    ss.TotalSales,
    dbo.CalculateSalesCategory(ss.TotalSales) AS SalesCategory
FROM
    SalesSummary ss
    INNER JOIN
    dbo.DimSalesTerritory sst ON ss.SalesTerritoryKey = sst.SalesTerritoryKey
WHERE
    ss.TotalSales > (
        SELECT AVG(TotalSales) * 1.5
FROM SalesSummary
    )
ORDER BY
    ss.TotalSales DESC;



--JSON
USE AdventureWorksDW2017;
GO
CREATE OR ALTER FUNCTION dbo.CalculateSalesCategory(@SalesAmount MONEY)
RETURNS NVARCHAR(50)
AS
BEGIN
    RETURN CASE
        WHEN @SalesAmount < 50000 THEN 'Low'
        WHEN @SalesAmount BETWEEN 50000 AND 100000 THEN 'Medium'
        ELSE 'High'
    END;
END;

GO

WITH
    SalesSummary
    AS
    (
        SELECT
            p.ProductKey,
            p.EnglishProductName,
            rs.SalesTerritoryKey,
            SUM(rs.SalesAmount) AS TotalSales
        FROM
            dbo.FactResellerSales rs
            INNER JOIN
            dbo.DimProduct p ON rs.ProductKey = p.ProductKey
        GROUP BY
        p.ProductKey, p.EnglishProductName, rs.SalesTerritoryKey
    )
SELECT
    sst.SalesTerritoryRegion,
    ss.EnglishProductName,
    ss.TotalSales,
    dbo.CalculateSalesCategory(ss.TotalSales) AS SalesCategory
FROM
    SalesSummary ss
    INNER JOIN
    dbo.DimSalesTerritory sst ON ss.SalesTerritoryKey = sst.SalesTerritoryKey
WHERE
    ss.TotalSales > (
        SELECT AVG(TotalSales) * 1.5
FROM SalesSummary
    )
ORDER BY
    ss.TotalSales DESC
FOR JSON PATH, ROOT('SalesPerformancePerProduct');



* * *

**Difficulty:** Complex 4

**Proposition:** Calculate net sales after applying geographical discounts for customers in the United States using a user-defined function for discount calculation.

**Function:** dbo.CalculateGeoDiscount

**Parameters:** @GeographyKey as INT (identifies the geographical region).

**Returns:** The discount percentage as DECIMAL(5, 2), determined by the geography key, with a default discount of 2% and special promotions increasing this to 5% for specified regions.

**Tables and Columns:**

dbo.DimCustomer: CustomerKey, FirstName, LastName, GeographyKey dbo.FactInternetSales: CustomerKey, SalesAmount dbo.DimGeography: GeographyKey, EnglishCountryRegionName

**Predicate:**

```
WHERE EnglishCountryRegionName LIKE '%United States%'

```

This predicate filters the dbo.DimGeography table to only return rows where the EnglishCountryRegionName contains 'United States', focusing the analysis on customers within the United States.

In [ ]:
USE AdventureWorksDW2017;
GO
CREATE OR ALTER FUNCTION dbo.CalculateGeoDiscount(@GeographyKey INT)
RETURNS DECIMAL(5, 2)
AS
BEGIN
    DECLARE @DiscountPercent DECIMAL(5, 2) = 2.0;
   

    IF @GeographyKey IN (1, 2, 3) 
        SET @DiscountPercent = 5.0;
  

    RETURN @DiscountPercent;
END;
GO

WITH
    SalesByCustomerAndGeo
    AS
    (
        SELECT
            cus.CustomerKey,
            cus.FirstName,
            cus.LastName,
            geo.GeographyKey,
            geo.EnglishCountryRegionName,
            SUM(fis.SalesAmount) AS TotalSalesAmount
        FROM dbo.DimCustomer cus
            INNER JOIN dbo.FactInternetSales fis ON cus.CustomerKey = fis.CustomerKey
            LEFT JOIN dbo.DimGeography geo ON cus.GeographyKey = geo.GeographyKey
        GROUP BY cus.CustomerKey, cus.FirstName, cus.LastName, geo.GeographyKey, geo.EnglishCountryRegionName
    )
SELECT
    s.FirstName + ' ' + s.LastName AS CustomerName,
    s.EnglishCountryRegionName,
    s.TotalSalesAmount,
    dbo.CalculateGeoDiscount(s.GeographyKey) AS DiscountPercent,
    s.TotalSalesAmount - (s.TotalSalesAmount * dbo.CalculateGeoDiscount(s.GeographyKey) / 100) AS NetSalesAfterDiscount
FROM SalesByCustomerAndGeo s
WHERE s.GeographyKey IN (
    SELECT DISTINCT GeographyKey
FROM dbo.DimGeography
WHERE EnglishCountryRegionName LIKE '%United States%'
)
ORDER BY s.TotalSalesAmount DESC;


--JSON
USE AdventureWorksDW2017;
GO
CREATE OR ALTER FUNCTION dbo.CalculateGeoDiscount(@GeographyKey INT)
RETURNS DECIMAL(5, 2)
AS
BEGIN
    DECLARE @DiscountPercent DECIMAL(5, 2) = 2.0;
   

    IF @GeographyKey IN (1, 2, 3) 
        SET @DiscountPercent = 5.0;
  

    RETURN @DiscountPercent;
END;
GO

WITH
    SalesByCustomerAndGeo
    AS
    (
        SELECT
            cus.CustomerKey,
            cus.FirstName,
            cus.LastName,
            geo.GeographyKey,
            geo.EnglishCountryRegionName,
            SUM(fis.SalesAmount) AS TotalSalesAmount
        FROM dbo.DimCustomer cus
            INNER JOIN dbo.FactInternetSales fis ON cus.CustomerKey = fis.CustomerKey
            LEFT JOIN dbo.DimGeography geo ON cus.GeographyKey = geo.GeographyKey
        GROUP BY cus.CustomerKey, cus.FirstName, cus.LastName, geo.GeographyKey, geo.EnglishCountryRegionName
    )
SELECT
    s.FirstName + ' ' + s.LastName AS CustomerName,
    s.EnglishCountryRegionName,
    s.TotalSalesAmount,
    dbo.CalculateGeoDiscount(s.GeographyKey) AS DiscountPercent,
    s.TotalSalesAmount - (s.TotalSalesAmount * dbo.CalculateGeoDiscount(s.GeographyKey) / 100) AS NetSalesAfterDiscount
FROM SalesByCustomerAndGeo s
WHERE s.GeographyKey IN (
    SELECT DISTINCT GeographyKey
FROM dbo.DimGeography
WHERE EnglishCountryRegionName LIKE '%United States%'
)
ORDER BY s.TotalSalesAmount DESC
FOR JSON PATH, ROOT('NetSalesAfterGeoDiscount');


* * *

**Difficulty:** Complex 5

**Proposition:** Apply supplier discounts based on total expected cost of orders placed in the year 2013.

**Function:** CalculateSupplierDiscount

**Parameters:** @TotalExpectedCost as DECIMAL(18,2) (the total expected cost of orders for a supplier).

**Returns:** The discount percentage as DECIMAL(5,2), determined by the total expected cost, with varying rates for different cost thresholds.

**Tables and Columns:**

Purchasing.PurchaseOrders: SupplierID, OrderDate, PurchaseOrderID Purchasing.PurchaseOrderLines: PurchaseOrderID, ExpectedUnitPricePerOuter, OrderedOuters

**Predicate:**

```
WHERE YEAR(po.OrderDate) = 2013

```

This predicate filters the Purchasing.PurchaseOrders table to only return orders placed in the year 2013, ensuring that the discount calculations are based on that year's expected costs.

In [ ]:
USE WideWorldimporters;
GO
CREATE OR ALTER FUNCTION CalculateSupplierDiscount(@TotalExpectedCost DECIMAL(18,2))
RETURNS DECIMAL(5,2) AS
BEGIN
    DECLARE @DiscountRate DECIMAL(5,2);
    IF @TotalExpectedCost > 50000 SET @DiscountRate = 7.5;
     ELSE IF @TotalExpectedCost > 20000 SET @DiscountRate = 5.0;
     ELSE SET @DiscountRate = 2.5;
    RETURN @DiscountRate;
END
GO

WITH
    ExpectedCosts
    AS
    (
        SELECT
            po.SupplierID,
            SUM(pol.ExpectedUnitPricePerOuter * pol.OrderedOuters) AS TotalExpectedCost
        FROM
            Purchasing.PurchaseOrders po
            JOIN Purchasing.PurchaseOrderLines pol ON po.PurchaseOrderID = pol.PurchaseOrderID
        WHERE
        YEAR(po.OrderDate) = 2013
        GROUP BY
        po.SupplierID
    ),
    Discounts
    AS
    (
        SELECT
            SupplierID,
            dbo.CalculateSupplierDiscount(TotalExpectedCost) AS DiscountPercentage
        FROM
            ExpectedCosts
    )

SELECT
    ec.SupplierID,
    ec.TotalExpectedCost,
    d.DiscountPercentage,
    CAST((ec.TotalExpectedCost * (1 - d.DiscountPercentage / 100)) AS DECIMAL(18,2)) AS TotalAfterDiscount
FROM
    ExpectedCosts ec
    JOIN Discounts d ON ec.SupplierID = d.SupplierID
ORDER BY
    ec.TotalExpectedCost DESC;


--JSON
USE WideWorldimporters;
GO
CREATE OR ALTER FUNCTION CalculateSupplierDiscount(@TotalExpectedCost DECIMAL(18,2))
RETURNS DECIMAL(5,2) AS
BEGIN
    DECLARE @DiscountRate DECIMAL(5,2);
    IF @TotalExpectedCost > 50000 SET @DiscountRate = 7.5;
     ELSE IF @TotalExpectedCost > 20000 SET @DiscountRate = 5.0;
     ELSE SET @DiscountRate = 2.5;
    RETURN @DiscountRate;
END
GO

WITH
    ExpectedCosts
    AS
    (
        SELECT
            po.SupplierID,
            SUM(pol.ExpectedUnitPricePerOuter * pol.OrderedOuters) AS TotalExpectedCost
        FROM
            Purchasing.PurchaseOrders po
            JOIN Purchasing.PurchaseOrderLines pol ON po.PurchaseOrderID = pol.PurchaseOrderID
        WHERE
        YEAR(po.OrderDate) = 2013
        GROUP BY
        po.SupplierID
    ),
    Discounts
    AS
    (
        SELECT
            SupplierID,
            dbo.CalculateSupplierDiscount(TotalExpectedCost) AS DiscountPercentage
        FROM
            ExpectedCosts
    )

SELECT
    ec.SupplierID,
    ec.TotalExpectedCost,
    d.DiscountPercentage,
    CAST((ec.TotalExpectedCost * (1 - d.DiscountPercentage / 100)) AS DECIMAL(18,2)) AS TotalAfterDiscount
FROM
    ExpectedCosts ec
    JOIN Discounts d ON ec.SupplierID = d.SupplierID
ORDER BY
    ec.TotalExpectedCost DESC
FOR JSON PATH, ROOT('SupplierDiscounts');





* * *

**Difficulty:** Complex 6

**Proposition:** Analyze sales data for a specific period, calculating total and average sales per category with a user-defined function for sales amount calculation.

**Function:** dbo.CalculateTotalSalesAmount

**Parameters:**

@Quantity as INT (quantity of the product sold), @UnitPrice as DECIMAL(18,2) (unit price of the product).

**Returns:** The total sales amount as DECIMAL(18,2), calculated by multiplying the quantity sold by the unit price.

**Tables and Columns:**

Fact.Sale: Quantity, Unit Price, Invoice Date Key, Customer Key, Stock Item Key Dimension.Customer: Customer Key, Category Dimension.\[Stock Item\]: Stock Item Key Predicate:

```
WHERE s.[Invoice Date Key] BETWEEN '2013-01-01' AND '2013-12-31'

```

This predicate filters the Fact.Sale table to only return rows where the Invoice Date Key is within the year 2013, focusing the analysis on this specific period.

In [ ]:
USE WideWorldImportersDW;
GO
CREATE OR ALTER FUNCTION dbo.CalculateTotalSalesAmount(@Quantity INT, @UnitPrice DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    RETURN @Quantity * @UnitPrice;
END;


GO

WITH
    CustomerSales
    AS
    (
        SELECT
            c.[Customer Key],
            c.Category,
            si.[Stock Item Key],
            s.[Invoice Date Key],
            dbo.CalculateTotalSalesAmount(s.Quantity, s.[Unit Price]) AS TotalSalesAmount
        FROM
            Fact.Sale s
            INNER JOIN Dimension.Customer c ON s.[Customer Key] = c.[Customer Key]
            LEFT JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
        WHERE
        s.[Invoice Date Key] BETWEEN '2013-01-01' AND '2013-12-31'
    ),
    AverageSalesPerCategory
    AS
    (
        SELECT
            Category,
            AVG(TotalSalesAmount) AS AvgSalesAmount
        FROM
            CustomerSales
        GROUP BY
        Category
    )

SELECT
    cs.Category,
    COUNT(DISTINCT cs.[Stock Item Key]) AS NumberOfDistinctItemsSold,
    SUM(cs.TotalSalesAmount) AS TotalSalesByCategory,
    (SELECT AvgSalesAmount
    FROM AverageSalesPerCategory
    WHERE Category = cs.Category) AS AverageSalesAmount
FROM
    CustomerSales cs
GROUP BY
    cs.Category;




--JSON
USE WideWorldImportersDW;
GO
CREATE OR ALTER FUNCTION dbo.CalculateTotalSalesAmount(@Quantity INT, @UnitPrice DECIMAL(18,2))
RETURNS DECIMAL(18,2)
AS
BEGIN
    RETURN @Quantity * @UnitPrice;
END;


GO

WITH
    CustomerSales
    AS
    (
        SELECT
            c.[Customer Key],
            c.Category,
            si.[Stock Item Key],
            s.[Invoice Date Key],
            dbo.CalculateTotalSalesAmount(s.Quantity, s.[Unit Price]) AS TotalSalesAmount
        FROM
            Fact.Sale s
            INNER JOIN Dimension.Customer c ON s.[Customer Key] = c.[Customer Key]
            LEFT JOIN Dimension.[Stock Item] si ON s.[Stock Item Key] = si.[Stock Item Key]
        WHERE
        s.[Invoice Date Key] BETWEEN '2013-01-01' AND '2013-12-31'
    ),
    AverageSalesPerCategory
    AS
    (
        SELECT
            Category,
            AVG(TotalSalesAmount) AS AvgSalesAmount
        FROM
            CustomerSales
        GROUP BY
        Category
    )

SELECT
    cs.Category,
    COUNT(DISTINCT cs.[Stock Item Key]) AS NumberOfDistinctItemsSold,
    SUM(cs.TotalSalesAmount) AS TotalSalesByCategory,
    (SELECT AvgSalesAmount
    FROM AverageSalesPerCategory
    WHERE Category = cs.Category) AS AverageSalesAmount
FROM
    CustomerSales cs
GROUP BY
    cs.Category
FOR JSON PATH, ROOT('TotalAverageSales');
